https://github.com/huggingface/evaluate/blob/0ca575d7aa0764ea646dcd5a27cb952e587ce9eb/metrics/perplexity/perplexity.py#L14

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.nn import CrossEntropyLoss

In [3]:
MODEL = "bigcode/gpt_bigcode-santacoder"
model = AutoModelForCausalLM.from_pretrained(MODEL).half().cuda()
tokenizer = AutoTokenizer.from_pretrained(MODEL, padding_side="left")

/home/claire.schlesinger/default/lib/python3.10/site-packages/torch/cuda/__init__.py:173: UserWarning: 
NVIDIA H100 PCIe with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75 sm_80 sm_86.
If you want to use the NVIDIA H100 PCIe GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [4]:
#TODO Confirm that this is correct, and if necessary, get someone at bigcode to fix it so it defaults to this
tokenizer.add_special_tokens({'pad_token': '<|endoftext|>'})

0

In [20]:
INPUT1 = "def foo(n):\n\treturn n + 1"
INPUT2 = "def foo(n: str):\n\treturn n + 1" 
INPUT3 = "# s is an integer\ndef foo(s):\n\treturn s + 1"
INPUT4 = "def foo(s: int):\n\treturn s + 1"
INPUT5 = "def foo(s):\n\treturn s + 1"
INPUT6 = "this is really long input that is not a valid python code. this is really long input that is not a valid python code"
input_toks = tokenizer([INPUT1, INPUT2, INPUT3, INPUT4, INPUT5, INPUT6], return_tensors="pt", padding=True).to(model.device)
input_toks

{'input_ids': tensor([[49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152,
         49152, 49152, 49152,   563,  6549,     7,    77,   399,   185,   184,
           640,   293,   385,   207,    16],
        [49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152,
         49152,   563,  6549,     7,    77,    25,   756,   399,   185,   184,
           640,   293,   385,   207,    16],
        [49152, 49152, 49152, 49152, 49152, 49152, 49152,     2,   308,   438,
           743,  5753,   185,   563,  6549,     7,    82,   399,   185,   184,
           640,   308,   385,   207,    16],
        [49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152,
         49152,   563,  6549,     7,    82,    25,   646,   399,   185,   184,
           640,   308,   385,   207,    16],
        [49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152,
         49152, 49152, 49152,   563,  6549,     7,    82,   399,   185,   184,
           640, 

### The Real Thing

In [ ]:
loss_fct = CrossEntropyLoss(reduction="none")
def compute_last_n_perplexity(r, last_n):
    """
      Computes the perplexity of the last n tokens of a result.

      Args:
        r: The result logits from the model.
        last_n: The last n tokens to compute perplexity from.

      Returns:
        The average peroplexity for each result in the batch over the last n tokens.
    """
    shift_logits = r[..., :-1, :].contiguous()
    shift_labels = input_toks.input_ids[..., 1:].contiguous()
    shift_attention_mask_batch = input_toks.attention_mask[..., 1:].contiguous()

    shift_logits = shift_logits[..., :-last_n, :]
    shift_labels = shift_labels[..., :-last_n]
    shift_attention_mask_batch = shift_attention_mask_batch[..., :-last_n]

    perplexity_batch = torch.exp(
                    (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
                    / shift_attention_mask_batch.sum(1)
                )

    return perplexity_batch

In [21]:
with torch.no_grad():
    r = model(**input_toks).logits